In [ ]:
import matplotlib.pyplot as plt
import torch
import cv2

In [1]:
dataset_location = '/home/mkim/Documents/RobotX/Buoy/yolov5/RobotX-Buoy-Detection-7'
# !python train.py --img 416 --batch 16 --epochs 150 --data {dataset_location}/data.yaml --weights yolov5s.pt --cache

In [1]:
weights_path = '/home/mkim/Documents/RobotX/Buoy/yolov5/runs/train/exp2/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', weights_path)

Using cache found in /home/mkim/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-14 Python-3.10.6 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12054MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
plot_path = "/home/mkim/Documents/RobotX/Buoy/yolov5/perception/plots/"
test_image_path = "/home/mkim/Documents/RobotX/Buoy/yolov5/RobotX-Buoy-Detection-7/test/images/9_jpg.rf.0742fd2b9691e8f00d5814ebac2fb097.jpg"
test_image_path = "/home/mkim/Documents/RobotX/Buoy/yolov5/RobotX-Buoy-Detection-7/test/images/92_jpg.rf.5143d76b7afac39c07ae9619003f8b92.jpg"

test_label_path = "/home/mkim/Documents/RobotX/Buoy/yolov5/RobotX-Buoy-Detection-7/test/labels/9_jpg.rf.0742fd2b9691e8f00d5814ebac2fb097.txt"
test_label_path = "/home/mkim/Documents/RobotX/Buoy/yolov5/RobotX-Buoy-Detection-7/test/labels/92_jpg.rf.5143d76b7afac39c07ae9619003f8b92.txt"


In [5]:
test_image = plt.imread(test_image_path)

plt.imshow(test_image)
plt.show()

plt.savefig(plot_path + 'test_image_square')

In [40]:
test_image.shape

(640, 640, 3)

In [6]:
with open(test_label_path) as f:
    lines = [bbox.strip() for bbox in f.readlines()]
    labels = [l.split(' ') for l in lines]  # cls, cx, cy, w, h
    print(labels)

[['1', '0.2203125', '0.628125', '0.03828125', '0.11328125'], ['2', '0.52734375', '0.61640625', '0.04453125', '0.13671875'], ['2', '0.75546875', '0.57421875', '0.0359375', '0.08828125'], ['0', '0.5984375', '0.58125', '0.0234375', '0.078125']]


In [7]:
label2class = {
    '0': 'blue-buoy',
    '1': 'pink-buoy',
    '2': 'white-buoy'
}
label2color = {
    '0': (0, 0, 255),
    '1': (255, 0, 0),
    '2': (0, 255, 0)
}

img_size = 640
scale = lambda x: int(img_size * float(x))

In [13]:
img = test_image.copy()

for y in labels:
    label, cx, cy, w, h = y
    cls, cx, cy, w, h = label2class[label], scale(cx), scale(cy), scale(w), scale(h)

    cv2.rectangle(img, (int(cx - w/2), int(cy - h/2)), ((int(cx+w/2)), int(cy+h/2)), label2color[label], 5)

img = cv2.resize(img, (1448, 568))
plt.imshow(img)
plt.show()
plt.savefig(plot_path + 'test_image_true')

In [24]:
detections = model(test_image)
detections

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 640x640 1 blue-buoy, 1 pink-buoy, 2 white-buoys
Speed: 1.7ms pre-process, 8.6ms inference, 1.1ms NMS per image at shape (1, 3, 640, 640)

In [33]:
predictions = detections.pred
predictions # x1, y1, y1, y2, confidence, cls

[tensor([[327.23633, 351.73798, 347.32861, 422.43695,   0.74727,   2.00000],
         [129.60585, 367.45319, 150.46104, 429.40387,   0.73723,   1.00000],
         [476.93851, 338.31360, 489.81955, 381.79858,   0.73380,   2.00000],
         [375.90073, 349.03168, 388.39383, 386.75604,   0.72975,   0.00000]], device='cuda:0')]

In [39]:
img = test_image.copy()

for pred in predictions[0]:
    x1, y1, x2, y2, confidence, label = pred
    label = str(int(label))
    x1, y1, x2, y2, cls = int(x1), int(y1), int(x2), int(y2), label2class[label]
    print(x1, x2, y1, y2, label2color[label])
    cv2.rectangle(img, (x1, y1), (x2, y2), label2color[label], 5)

img = cv2.resize(img, (1448, 568))
plt.imshow(img)
plt.show()
plt.savefig(plot_path + 'test_image_pred')

327 347 351 422 (0, 255, 0)
129 150 367 429 (255, 0, 0)
476 489 338 381 (0, 255, 0)
375 388 349 386 (0, 0, 255)


In [47]:
plt.imshow(img)